In [52]:
import pandas as pd 
import os
from utils.explanations import minmax_norm
from utils.utils import read_dataset
from utils.constants import CAM_LAYERS
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
from utils.explanations import calculate_cam_attributions
import matplotlib.pyplot as plt

import tensorflow_addons as tfa
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils import CustomObjectScope
def load_model(root_dir, archive_name, dataset_name, classifier, data_source): 
    with CustomObjectScope({'InstanceNormalization':tfa.layers.InstanceNormalization()}):
        model_path = f'{root_dir}/results/{archive_name}/{dataset_name}/' \
                                                + f'{classifier.split("_")[0]}/{classifier}/{data_source}/' \
                                                + f'best_model.hdf5'
        model =keras.models.load_model(model_path ,compile=False)
        return model



root_dir ='G:/Meine Ablage/master thesis/code/xai-tsc'
archive_name = 'ucr'
classifier = 'fcn_1.0'
dataset_name = 'GunPoint'
data_source = 'original'

data = read_dataset(root_dir, 'ucr', dataset_name,  data_source, 1)[dataset_name]

base_model = load_model(root_dir, archive_name, dataset_name, classifier, data_source)

#G:\Meine Ablage\master thesis\code\xai-tsc\results\ucr\GunPoint\fcn\fcn_mt_dense_0.5d

classifier = 'fcn_mt_dense_0.0'
data_source = 'fcn_raw'
dataset_name = 'GunPoint'
data_source = 'fcn_raw_mse'

sigmoid_model =  load_model(root_dir, archive_name, dataset_name, classifier, data_source)

In [53]:
from utils.constants import CAM_LAYERS
from utils.explanations import get_layer_index


bact = CAM_LAYERS[classifier.split("_")[0]]["gap_layer"]
bact = get_layer_index(base_model, bact)

sact = CAM_LAYERS[classifier.split("_")[0]]["gap_layer"]
sact = get_layer_index(sigmoid_model, sact)

bnew_input_layer = base_model.inputs
bnew_output_layer = [base_model.layers[bact].output]
bew_feed_forward = keras.backend.function(bnew_input_layer, bnew_output_layer)


sew_input_layer = sigmoid_model.inputs
sew_output_layer = [sigmoid_model.layers[sact].output]
sew_feed_forward = keras.backend.function(sew_input_layer, sew_output_layer)


ts = data[0]

[bconv_out] = bew_feed_forward([ts])
[sconv_out] = sew_feed_forward([ts])

In [54]:
mean_corr = 0
highest_filter_corr = 0
for i in range(128):
    #print(np.corrcoef(bconv_out[0][i],sconv_out[0][i]))
    mean_corr += np.corrcoef(bconv_out[0][i],sconv_out[0][i])
    for j in range(128):
        #print(np.corrcoef(bconv_out[0][i],sconv_out[0][i]))
        if np.corrcoef(bconv_out[0][i],sconv_out[0][i])[0,1] > highest_filter_corr:
            highest_filter_corr = np.corrcoef(bconv_out[0][i],sconv_out[0][i])[0,1] 


print(mean_corr / 128, highest_filter_corr)

[[ 1.         -0.02135652]
 [-0.02135652  1.        ]] 0.08917169168714735
